# Forest of Doom
### By Ryan Dickson
### Edited by Mohinder Dick
> References go here for presentation version. 

In [14]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
from dateutil.parser import parse
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, HashingTF, VectorAssembler, IDF
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

## Set up environment depend variables

In [2]:
# Set up the path of the files and environment va

if sc._conf.get('spark.master') == 'yarn-client' or sc._conf.get('spark.master') == 'yarn-cluster':
    urn = 'hdfs://sparkdl04:8020/palooza/data/visit_train_panda.csv'
    urnTest = 'hdfs://sparkdl04:8020/palooza/data/validate/visit_test_panda.csv'
    
    # Set up the path of the python pyspark should use
    #os.environ['PYSPARK_PYTHON'] = '/opt/anaconda/bin/python'
else:
    urn = 'file:///C:/Users/dickm/Documents/Projects/ML/Source/UPMC/Pharmacy/visit_train_panda.csv'
    urnTest = 'file:///C:/Users/dickm/Documents/Projects/ML/Source/UPMC/Pharmacy/visit_test_panda.csv'
    #os.environ['PYSPARK_PYTHON'] = 'C:\Users\dickm\AppData\Local\Continuum\Anaconda2\python.exe'

# Used avoid issues in windows when reading text files.
partitionNumber = 500

print('Train file path: %s.\nTest file path: %s.\nPyspark path: %s.' % (urn, urnTest, os.environ.get('PYSPARK_PYTHON')))

Train file path: file:///C:/Users/dickm/Documents/Projects/ML/Source/UPMC/Pharmacy/visit_train_panda.csv.
Test file path: file:///C:/Users/dickm/Documents/Projects/ML/Source/UPMC/Pharmacy/visit_test_panda.csv.
Pyspark path: None.


## Verify Spark

This launches the notebook and provides the spark context in the variable *sc*. You can use the context to preview the configuration.

In [3]:
print sc._conf.getAll()
test = sc.parallelize(range(1000))
print '\nsum of 1 to 1000: ', test.reduce(lambda a, b: a+b )

[(u'spark.executor.instances', u'6'), (u'spark.executor.memory', u'512m'), (u'spark.rdd.compress', u'True'), (u'spark.master', u'local[3]'), (u'spark.serializer.objectStreamReset', u'100'), (u'spark.submit.deployMode', u'client'), (u'spark.app.name', u'PySparkShell')]

sum of 1 to 1000:  499500


## Get the training data

I added parallelism due to issue on windows

In [7]:
chargesRDD = sc.textFile(urn, partitionNumber)

#Get a new RDD with map function and lambda keyword. Remove header row.
header = chargesRDD.take(1)[0]
chargesRDD = chargesRDD.filter(lambda line: line!=header)
chargesRDDSplit = chargesRDD.map(lambda line: line.replace('"', '').split(','))
chargesRDDSplit.cache()

PythonRDD[7] at RDD at PythonRDD.scala:43

## Get the test data

Mo knows where it's at!

In [8]:
chargesRDDTEST = sc.textFile(urnTest, partitionNumber)
   
chargesRDDTEST = chargesRDD.filter(lambda line: line!=header)
chargesRDDSplitTEST = chargesRDD.map(lambda line: line.replace('"', '').split(','))
chargesRDDSplitTEST.cache()

PythonRDD[10] at RDD at PythonRDD.scala:43

In [10]:
#look at the data
print ('The number of training records is %s.\nThe number of test records is %s' % 
       (chargesRDD.count(), chargesRDDTEST.count()))
print ('First row of train:\n%s\nThe first row of test:\n%s\n' % (chargesRDD.take(1), chargesRDDTEST.take(1)))

The number of training records is 279808.
The number of test records is 279808
First row of train:
[u'"HdBgCT1YkEl14280","SHY",436,"I",77,"W","M","MS","10/07/2014","10/10/2014",3,"197.7","SECOND MALIG NEO LIVE","1",10028']
The first row of test:
[u'"HdBgCT1YkEl14280","SHY",436,"I",77,"W","M","MS","10/07/2014","10/10/2014",3,"197.7","SECOND MALIG NEO LIVE","1",10028']



### Raw Features

* VisitID - Identifier for patient visit.
* Hospital - Admitting hospital.
* Dept_Code - department code.
* PaymentType - I am guessing a payment type for visit.
* Age - Age of the patient in years.
* Race - De-identified race of the patient.
* Gender - Gender ("M" - male, "F" - female)
* FC - ?
* ArriveDate - Date of admission. 
* DischargeDate - Date of discharge
* LOS - length of patient stay in days.
* DXCODE - Diagnosis code.
* Description - Description of diagnosis
* DispenseID - ?
* DOC - ?

## Build Random Forest Model
### We want to predict the length of stay (LOS) given the patient demographics, dxcode and deptcode, and day of the week admitted.

Assumptions of feature relevance...
* Dept_code categorical, some departments would have more serious patients than others
* Day of the week, Patients admitted over weekend may require a longer length of stay to be seen by necessary staff
* Dxcode, multiple per patient, may need to use PCA to reduce number (PCA is not always good before random forest, see references)
* Demographics: age, gender and race 


We go through the following pipeline:
* Encode/Extract features
* Train the model
* Evaluate the model on unseen data
* Draw conclusions and make recommendations

We should not need to normalize the features for Random Forest, but bucketing may be needed to help with outliers

In [11]:
dxcodeRDD = chargesRDDSplit.map(lambda line: (line[0], line[11])).groupByKey().distinct().cache()
dxCodes = dxcodeRDD.values().flatMap(list).distinct()
dxCodeCount = dxCodes.count()

In [15]:
def merge(x, y):
    
    if x is not None: 
        x['dxCount'] += 1
        if 'dxcode' in x:
            x['dxcode'] = list(set(x['dxcode'] + y['dxcode']))
        else:
            x = y
    else:
        x = y
    return x


def mapAndFold(rdd):
    return rdd.map(lambda line: (line[0], Row(
             los=float(line[10]),
             age=int(line[4]),
             hospital_visit=line[1],
             dept_code=line[2],
             race=line[5], 
             gender_female=1 if line[6]=='F' else 0,    #Encode gender as boolean
             dxcode=[line[11]],
             admit_day=parse(line[8]).weekday(),
             admit_month=parse(line[8]).month,
             dxCount=1,
             fc=line[7]
            ))).foldByKey(None, merge)


In [13]:
chargesByVisitRDD = mapAndFold(chargesRDDSplit)
#df = chargesByVisitRDD.values().toDF()

C:\Applications\spark-1.6.1-bin-hadoop2.6\python\pyspark\sql\context.py:259: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "
You must build Spark with Hive. Export 'SPARK_HIVE=true' and run build/sbt assembly


Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.hive.HiveContext.
: java.lang.RuntimeException: java.lang.NullPointerException
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.ClientWrapper.<init>(ClientWrapper.scala:204)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:238)
	at org.apache.spark.sql.hive.HiveContext.executionHive$lzycompute(HiveContext.scala:218)
	at org.apache.spark.sql.hive.HiveContext.executionHive(HiveContext.scala:208)
	at org.apache.spark.sql.hive.HiveContext.functionRegistry$lzycompute(HiveContext.scala:462)
	at org.apache.spark.sql.hive.HiveContext.functionRegistry(HiveContext.scala:461)
	at org.apache.spark.sql.UDFRegistration.<init>(UDFRegistration.scala:40)
	at org.apache.spark.sql.SQLContext.<init>(SQLContext.scala:330)
	at org.apache.spark.sql.hive.HiveContext.<init>(HiveContext.scala:90)
	at org.apache.sp

In [19]:
chargesByVisitRDD.take(2)

[(u'nizzRT/Dabr94244',
  {'admit_day': 0,
   'admit_month': 9,
   'age': 64,
   'dept_code': u'811',
   'dxCount': 19,
   'dxcode': [u'530.85',
    u'276.2',
    u'276.51',
    u'272.4',
    u'263.9',
    u'414.01',
    u'534.40',
    u'285.1',
    u'276.8',
    u'537.4',
    u'V12.71',
    u'729.1',
    u'327.23',
    u'244.9',
    u'496',
    u'V12.55',
    u'346.90',
    u'530.81',
    u'V12.51'],
   'fc': u'UM',
   'gender_female': 1,
   'hospital_visit': u'MCH',
   'los': 2.0,
   'race': u'W'}),
 (u'afZQXxAQW+D+4218',
  {'admit_day': 2,
   'admit_month': 8,
   'age': 31,
   'dept_code': u'392',
   'dxCount': 8,
   'dxcode': [u'300.00',
    u'112.1',
    u'V12.55',
    u'276.51',
    u'338.29',
    u'493.90',
    u'787.01',
    u'V44.2'],
   'fc': u'M',
   'gender_female': 1,
   'hospital_visit': u'SHY',
   'los': 2.0,
   'race': u'W'})]

In [16]:
df = chargesByVisitRDD.values().toDF()

You must build Spark with Hive. Export 'SPARK_HIVE=true' and run build/sbt assembly


Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.hive.HiveContext.
: java.lang.RuntimeException: java.lang.NullPointerException
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.ClientWrapper.<init>(ClientWrapper.scala:204)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:238)
	at org.apache.spark.sql.hive.HiveContext.executionHive$lzycompute(HiveContext.scala:218)
	at org.apache.spark.sql.hive.HiveContext.executionHive(HiveContext.scala:208)
	at org.apache.spark.sql.hive.HiveContext.functionRegistry$lzycompute(HiveContext.scala:462)
	at org.apache.spark.sql.hive.HiveContext.functionRegistry(HiveContext.scala:461)
	at org.apache.spark.sql.UDFRegistration.<init>(UDFRegistration.scala:40)
	at org.apache.spark.sql.SQLContext.<init>(SQLContext.scala:330)
	at org.apache.spark.sql.hive.HiveContext.<init>(HiveContext.scala:90)
	at org.apache.sp

In [ ]:
chargesByVisitRDDTEST = mapAndFold(chargesRDDSplitTEST)
dfTEST = chargesByVisitRDD.values().toDF()

In [ ]:
stringIndexers = [
    StringIndexer(inputCol="hospital_visit", outputCol="hospitalIndex"),
    StringIndexer(inputCol="dept_code", outputCol="deptIndex"),
    StringIndexer(inputCol="race", outputCol="raceIndex")
]

hashingTF = HashingTF(numFeatures=2*dxCodeCount, inputCol="dxcode", outputCol="dxCodes")
idf = IDF(inputCol="dxCodes", outputCol="idfDxCodes", minDocFreq=10)

mungePipeline = Pipeline(stages=stringIndexers + [hashingTF, idf])

mungingModel = mungePipeline.fit(df)
trainingData = mungingModel.transform(df)

In [ ]:
testingData = mungingModel.transform(dfTEST)

## Munge it!
String Indexers Need to have all values, so will need to fit combined traing and test data if unseen labels are present


In [ ]:
assembler = VectorAssembler(
   inputCols=["age", 
              "deptIndex", 
              "gender_female",
              "raceIndex",
              "hospitalIndex",
              "admit_day",
              "admit_month",
              "dxCount",
              "idfDxCodes"
             ],
    outputCol="features")

transformedTrainingDF = assembler.transform(trainingData).select('features','los')

In [ ]:
transformedTestingDF = assembler.transform(testingData).select('features','los')

### Train Model
Now we generate the training and test data. We use the ***seed*** function to ensure a repeatable split of the data between runs.

In [ ]:
rf = RandomForestRegressor(featuresCol="features", labelCol="los",maxBins=1000, seed=1234)

#Magic Numbers?
rf.setNumTrees(100) 
rf.setMaxDepth(10) # Max of Spark is 30
rf.setMinInstancesPerNode(5)
rf.setFeatureSubsetStrategy('all')

model = rf.fit(transformedTrainingDF)

### Evaluate Model
We evaluate the model on the unseen dataset that was not used to train the model. For reference here is the stats for los in the training data

In [ ]:
df.describe('los').show()

In [ ]:
# Select (prediction, true label) and compute test error
predictions = model.transform(transformedTestingDF).select('los','prediction')
evaluator = RegressionEvaluator(
    labelCol="los", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)